# Generate experimental features -  experiments 21-10

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import MEAutility as mu
from pprint import pprint
import time
import numpy as np
import neuroplotlib as nplt

import multimodalfitting as mf

import sys
import shutil

from pathlib import Path
import os
from datetime import datetime

%matplotlib notebook

In [ ]:
base_folder = Path("../../..")

In [ ]:
cell_name = "cell1_211011_3436"  # "cell1_211006_3148" | "cell1_211011_3436"

In [ ]:
data_folder = base_folder / "experimental_data" / cell_name

cell_models_folder = base_folder / "cell_models"
model_folder = cell_models_folder / cell_name
model_name = cell_name
fitting_folder = model_folder / "fitting"
extra_folder = data_folder / "extracellular"

## 1) Generate features and protocols with BluePyEfe

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

In [ ]:
with (extra_folder / "probe.json").open() as f:
    probe_info = json.load(f)
    probe_info["pitch"] = 17.5
    probe = mu.return_mea(info=probe_info)

In [ ]:
eap = np.load(extra_folder / "template.npy")
locations = np.load(extra_folder / "locations.npy")
fs = 20000

In [ ]:
ax_eap = mu.plot_mea_recording(eap, probe)

In [ ]:
efeatures_output_directory = fitting_folder / "efeatures"
ephys_dir = data_folder / "patch_data"

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

In [ ]:
from multimodalfitting.efeatures_extraction import build_wcp_metadata, wcp_reader, get_ecode_targets

In [ ]:
# select files for different repetitions

In [ ]:
# define timings for this experiment
ecodes_cell_timings = {
    "IDthres": {
        'ton': 400,
        'toff': 670
    },
    "firepattern": {
        'ton': 500,
        'toff': 4100
    },
    "IV": {
        'ton': 400,
        'toff': 3400
    },
    "IDrest": {
        'ton': 400,
        'toff': 1750
    },
    "APWaveform": {
        'ton': 350,
        'toff': 400
    },
    "HyperDepol": {
        'ton': 400,
        'toff': 1120,
        'tmid': 850
    },
    "sAHP": {
        'ton': 400,
        'toff': 1325,
        'tmid': 650,
        'tmid2': 875
    },
    "PosCheops": {
        'ton': 1000,
        't1': 9000,
        't2': 10500,
        't3': 14500,
        't4': 16000,
        'toff': 18660
    }
}

In [ ]:
if cell_name == "cell1_211006_3148":
    runs = [1, 2, 3, 4, 5]  # run1 --> different rheobase
elif cell_name == "cell1_211011_3436":
    runs = [3, 4, 5, 6]
    
ecode_names = list(ecodes_cell_timings.keys())

files_list = []

for run in runs:
    rep_dict = {}
    for ecode in ecode_names:
        for patch_file in ephys_dir.iterdir():
            if f"run{run}" in patch_file.name and ecode.lower() in patch_file.name:
                rep_dict[ecode] = patch_file
    files_list.append(rep_dict)

In [ ]:
files_list

In [ ]:
files_metadata = build_wcp_metadata(cell_id=cell_name, 
                                    files_list=files_list, 
                                    ecode_timings=ecodes_cell_timings, 
                                    repetition_as_different_cells=False)
pprint(files_metadata[cell_name])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=wcp_reader
)

In [ ]:
# define target features for different protocols
targets_legacy = get_ecode_targets(ecodes_cell_timings)

In [ ]:
targets_legacy

In [ ]:
global_tolerance = 30
for protocol, prot_dict in targets_legacy.items():
    prot_dict["tolerances"] = [global_tolerance]

In [ ]:
targets = convert_legacy_targets(targets_legacy)

In [ ]:
pprint(targets)

In [ ]:
t_start = time.time()
extract_efeatures_at_targets(
    cells, 
    targets,
    efel_settings={'interp_step': 0.1}
)
t_stop = time.time()
print(f"Elapsed time {t_stop - t_start}")

In [ ]:
if cell_name == "cell1_211006_3148":
    majority = 0.4
else:
    majority = 0.2

In [ ]:
compute_rheobase(
    cells,
    protocols_rheobase=['IDthres'],
    rheobase_strategy="majority",
    majority=majority
)

In [ ]:
print(f"Cell rheobase: {cells[0].rheobase} nA")

In [ ]:
protocols = group_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=efeatures_output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

In [ ]:
pprint(efeatures.keys())

## 2) Convert to BPO format and append extra features


In [ ]:
from multimodalfitting.efeatures_extraction import compute_extra_features, convert_to_bpo_format,\
    append_extrafeatures_to_json

In [ ]:
protocols_of_interest = ["IDrest_150", "IDrest_250", "IDrest_300", "IV_-100", "IV_-20", "APWaveform_290"]

# remove sag features from IV_-20
exclude_features = {"IV_-20": ['sag_amplitude', 'sag_ratio1', 'sag_ratio2']}
if cell_name == "cell1_211006_3148":
    exclude_features.update({"IDrest_150": ['burst_number'],
                             "IDrest_250": ['burst_number'],
                             "IDrest_300": ['burst_number']})

in_protocol_path = efeatures_output_directory / "protocols.json"
in_efeatures_path = efeatures_output_directory / "features.json"

In [ ]:
exclude_features

## Threshold EAP and extract featrues

In [ ]:
thresh_uV = 5

In [ ]:
amp_eap = np.ptp(eap, 1)
above_tr = np.where(amp_eap > thresh_uV)
eap_above = eap[above_tr]
extra_features_above = compute_extra_features(eap_above, fs, upsample=10)
probe_info = probe.info
probe_info["pos"] = locations[above_tr].tolist()
probe_above = mu.return_mea(info=probe_info)

In [ ]:
ax_eap = mu.plot_mea_recording(eap_above, probe_above)

In [ ]:
# save thresholded probe_above.json and template
json.dump(probe_above.info, (efeatures_output_directory / "probe_BPO.json").open("w"))
np.save(efeatures_output_directory / "template_BPO.npy", eap_above)

In [ ]:
extra_features = compute_extra_features(eap_above, fs, upsample=10)

In [ ]:
pprint(extra_features.keys())

In [ ]:
# plot one extra features
for feat in extra_features.keys():
    f = mf.plot_feature_map_w_colorbar(extra_features[feat], probe_above, 
                                       feature_name=feat, label="a.u.")

### a) All channels

In [ ]:
out_protocol_path_all = efeatures_output_directory / "protocols_BPO_all.json"
out_efeatures_path_all = efeatures_output_directory / "features_BPO_all.json"

In [ ]:
protocols_dict_all, efeatures_dict_all = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                               out_protocol_path_all, 
                                                               out_efeatures_path_all, 
                                                               protocols_of_interest=protocols_of_interest, 
                                                               exclude_features=exclude_features,
                                                               std_from_mean=None)

In [ ]:
pprint(efeatures_dict_all)

In [ ]:
channel_ids_all = None
single_channel_features = False

In [ ]:
extrafeatures_dict_all = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                      efeatures_dict=efeatures_dict_all,
                                                      efeatures_path=out_efeatures_path_all, 
                                                      channel_ids=channel_ids_all,
                                                      single_channel_features=single_channel_features, 
                                                      std_from_mean=None)

In [ ]:
pprint(extrafeatures_dict_all["extra"]["IDrest_300"]["MEA"])

### b) Select single channels

In [ ]:
cell = mf.create_experimental_model(model_name=model_name,
                                    release=True)
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe_above, mechs_folders=model_folder)

In [ ]:
out_protocol_path_single = efeatures_output_directory / "protocols_BPO_single.json"
out_efeatures_path_single = efeatures_output_directory / "features_BPO_single.json"

In [ ]:
protocols_dict_single, efeatures_dict_single = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                     out_protocol_path_single, 
                                                                     out_efeatures_path_single, 
                                                                     protocols_of_interest=protocols_of_interest,
                                                                     exclude_features=exclude_features,
                                                                     std_from_mean=None)

In [ ]:
ax_eap = mu.plot_mea_recording(eap_above / np.max(np.abs(eap_above), axis=1, keepdims=True), probe_above)

In [ ]:
channel_ids_single = mf.select_single_channels(cell, sim, probe_above)

In [ ]:
channel_ids_single

In [ ]:
single_channel_features = True
std_from_mean = 0.05

In [ ]:
extrafeatures_dict_single = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                         efeatures_dict=efeatures_dict_single,
                                                         efeatures_path=out_efeatures_path_single, 
                                                         channel_ids=channel_ids_single,
                                                         single_channel_features=single_channel_features, 
                                                         std_from_mean=std_from_mean)

In [ ]:
pprint(extrafeatures_dict_single["extra"]["IDrest_300"]["MEA"])

### c) Select channel sections

In [ ]:
out_protocol_path_sections = efeatures_output_directory / "protocols_BPO_sections.json"
out_efeatures_path_sections = efeatures_output_directory / "features_BPO_sections.json"

In [ ]:
protocols_dict_sections, efeatures_dict_sections = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                         out_protocol_path_sections, 
                                                                         out_efeatures_path_sections, 
                                                                         protocols_of_interest=protocols_of_interest,
                                                                         exclude_features=exclude_features,
                                                                         std_from_mean=None)

In [ ]:
ax_eap = mu.plot_mea_recording(eap_above / np.max(np.abs(eap_above), axis=1, keepdims=True), probe_above)

In [ ]:
channel_ids_sections = mf.select_mea_sections(cell, sim, probe_above)

In [ ]:
channel_ids_sections

In [ ]:
single_channel_features = False
std_from_mean = None

In [ ]:
extrafeatures_dict_sections = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                           efeatures_dict=efeatures_dict_sections,
                                                           efeatures_path=out_efeatures_path_sections, 
                                                           channel_ids=channel_ids_sections,
                                                           single_channel_features=single_channel_features, 
                                                           std_from_mean=std_from_mean)

In [ ]:
pprint(extrafeatures_dict_sections["extra"]["IDrest_300"])